# Upstream Data Upload Guide

## Overview

This guide demonstrates how to authenticate with the Upstream API and upload sensor data using CSV files for environmental monitoring campaigns.

## What You Can Do

The Upstream API allows you to:
- Authenticate and obtain access tokens
- Upload sensor definitions and measurement data
- Manage environmental monitoring campaigns
- Query and retrieve measurement data

## Prerequisites

- Valid Upstream account credentials
- Python 3.7+ with `requests` library installed
- CSV files with sensor and measurement data formatted correctly

## Installation

```bash
pip install requests
```

## Quick Start

1. **Authenticate** with the API to get your access token
2. **Prepare your CSV files** following the required format
3. **Upload your data** using the provided functions
4. **Monitor the results** and verify successful upload

In [18]:

import requests
import json
import getpass
import os
from typing import Dict, Any, Optional, List


## 1. Authentication

First, we need to authenticate with the Upstream API to obtain an access token.


In [3]:

def authenticate_upstream(base_url: str = "https://upstream-dso.tacc.utexas.edu/dev") -> str:
    """
    Authenticate with Upstream API and return access token.
    
    Args:
        base_url: Base URL for the Upstream API (dev or prod)
        
    Returns:
        Access token string
        
    Raises:
        Exception: If authentication fails
    """
    auth_url = f"{base_url}/api/v1/token"
    
    print("=== Upstream API Authentication ===")
    credentials = {
        "username": input("Username: "),
        "password": getpass.getpass("Password: ")
    }
    
    try:
        response = requests.post(auth_url, data=credentials)
        response.raise_for_status()
        
        token = response.json().get("access_token")
        if not token:
            raise Exception("No access token in response")
            
        print("✅ Authentication successful!")
        return token
        
    except requests.exceptions.RequestException as e:
        raise Exception(f"Authentication failed: {e}")

# Get authentication token
token = authenticate_upstream()


=== Upstream API Authentication ===
✅ Authentication successful!


In [5]:
def make_authenticated_request(
    method: str,
    url: str,
    token: str,
    json: Optional[Dict] = None,
    files: Optional[Dict] = None,
    params: Optional[Dict] = None
) -> requests.Response:
    """
    Make an authenticated HTTP request to the Upstream API.
    
    Args:
        method: HTTP method (GET, POST, PUT, DELETE, etc.)
        url: Full URL for the request
        token: Authentication token
        json: JSON data for the request body
        files: Files for multipart upload
        params: URL parameters
        
    Returns:
        Response object from the request
        
    Raises:
        requests.exceptions.HTTPError: If the request fails
    """
    headers = {
        "Authorization": f"Bearer {token}",
    }
    
    # Don't set Content-Type for file uploads (requests will set it automatically)
    if files is None:
        headers["Content-Type"] = "application/json"
    
    try:
        response = requests.request(
            method=method.upper(),
            url=url,
            headers=headers,
            json=json,
            files=files,
            params=params,
            timeout=300  # 5 minute timeout for large file uploads
        )
        
        # Raise an exception for bad status codes
        response.raise_for_status()
        
        return response
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e}")
        print(f"Response content: {response.text}")
        raise
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
        raise

## 2. Helper Functions for API Requests


In [14]:
def create_campaign(
     campaign_data:str,
    
    token: str,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Create a new campaign.
    
    Args:
        name: Campaign name
        description: Campaign description
        allocation: TACC allocation identifier (required)
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing the created campaign data with ID
    """
    url = f"{base_url}/api/v1/campaigns"    
    response = make_authenticated_request(
        method="POST",
        url=url,
        token=token,
        json=campaign_data
    )
    
    result = response.json()
    print(f"✅ Campaign created successfully!")
    print(f"Campaign ID: {result.get('id')}")

    
    return result



### Creating Campaigns

Before uploading CSV data, you need to create a campaign to organize your data collection project. A campaign serves as the top-level container for all related monitoring activities.

#### Campaign Requirements

**Required Fields:**
- `name`: Descriptive name for your data collection project
- `description`: Detailed description of the campaign's purpose and scope

#### Campaign Best Practices

🎯 **Naming Conventions:**
- Use descriptive, unique names that clearly identify the project
- Include dates, locations, or project codes for easy identification
- Examples: "Austin Air Quality 2024", "Hurricane Harvey Recovery Monitoring"

📝 **Descriptions:**
- Provide detailed context about the campaign's objectives
- Include information about duration, scope, and expected outcomes
- Mention any relevant research or operational goals

In [ ]:
def load_and_create_campaign(
    config_path: str = "campaigns/campaign.json",
    token: str = None,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Load campaign configuration from JSON and create the campaign.
    
    Args:
        config_path: Path to the campaign configuration JSON file
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing the created campaign data with ID
    """
    # Load configuration
    with open(config_path) as campaign_data:
        campaign_json = json.loads(campaign_data.read())

    # Validate required fields
    required_fields = ["name", "description"]
    for field in required_fields:
        if field not in campaign_json:
            raise ValueError(f"Missing required field '{field}' in campaign config")
    
    # Display configuration summary
    print(f"📋 Campaign Configuration Summary:")
    print(f"  Name: {campaign_json['name']}")
    print(f"  Description: {campaign_json['description'][:100]}...")
    
    if "metadata" in campaign_json:
        metadata = campaign_json["metadata"]
        print(f"  Project Lead: {metadata.get('project_lead', 'N/A')}")
        print(f"  Institution: {metadata.get('institution', 'N/A')}")
    
    # Create the campaign
    campaign = create_campaign(
        campaign_data=campaign_json,
        token=token,
        base_url=base_url
    )
    return campaign

# Usage example
print("=== Creating Campaign from Configuration ===")
try:
    campaign = load_and_create_campaign(
        config_path="campaigns/campaign.json",
        token=token
    )
    
    campaign_id = campaign['id']
    print(f"\n🎉 Campaign setup complete!")
    print(f"Campaign ID: {campaign_id}")
    
except FileNotFoundError as e:
    print(f"❌ Configuration file error: {e}")
    print("💡 Please create a campaigns/campaign.json file with your campaign details")
except ValueError as e:
    print(f"❌ Configuration error: {e}")
except Exception as e:
    print(f"❌ Campaign creation failed: {e}")


=== Creating Campaign from Configuration ===
📋 Campaign Configuration Summary:
  Name: Beaumont Stream Gauge
  Description: Beaumont Stream Gauge Campaign...
✅ Campaign created successfully!
Campaign ID: 10

🎉 Campaign setup complete!
Campaign ID: 10


### Creating Stations

Once you have a campaign, you need to create stations within it. Stations represent specific monitoring locations where sensors collect data.

#### Station Requirements

**Required Fields:**
- `campaign_id`: ID of the parent campaign (must exist)
- `name`: Unique name for the monitoring station
- `description`: Details about the station location and purpose
- `latitude`: Decimal degrees (e.g., 30.2672)
- `longitude`: Decimal degrees (e.g., -97.7431)

#### Station Best Practices

📍 **Location Data:**
- Ensure coordinates are in decimal degrees format
- Use WGS84 coordinate system (standard GPS coordinates)
- Verify coordinates are accurate for your monitoring location
- Test coordinates in mapping software before creating stations

🏷️ **Station Naming:**
- Use descriptive names that indicate location or purpose
- Include geographic references or landmarks
- Examples: "River Bridge Station", "Industrial District Monitor"

📝 **Station Descriptions:**
- Describe the physical location and surroundings
- Note any special characteristics or constraints
- Include installation details or access information

#### Alternative: Web Interface for Stations

If you prefer using the web interface:

1. **Navigate to Campaign:**
   - Go to your created campaign in the web portal
   - Access the campaign details page

2. **Create Station:**
   - Go to the "Stations" section within the campaign
   - Click "Add Station"
   - Provide station details and coordinates
   - Save to get your Station ID

3. **Note the Station ID:**
   - Copy the Station ID for use in data uploads


💡 **Pro Tip:** Save your campaign and station IDs in a configuration file or notebook cell for easy reuse across multiple data uploads.

In [20]:
def create_station(
    station_data: Dict[str, Any],
    campaign_id: int,
    token: str,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Create a new station within a campaign.
    
    Args:
        station_data: Dictionary containing station information
        campaign_id: ID of the parent campaign
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing the created station data with ID
    """
    url = f"{base_url}/api/v1/campaigns/{campaign_id}/stations"
    
    response = make_authenticated_request(
        method="POST",
        url=url,
        token=token,
        json=station_data
    )
    
    result = response.json()
    print(f"✅ Station created successfully!")
    print(f"Station ID: {result.get('id')}")
    print(f"Station Name: {station_data.get('name')}")
    print(f"Project ID: {station_data.get('projectid')}")
    print(f"Contact: {station_data.get('contact_name')}")
    
    return result

def load_station_config(config_path: str = "stations/station.json") -> Dict[str, Any]:
    """
    Load station configuration from JSON file.
    
    Args:
        config_path: Path to the station configuration JSON file
        
    Returns:
        Dictionary containing station configuration data
    """
    try:
        with open(config_path, 'r', encoding='utf-8') as file:
            config = json.load(file)
            print(f"📄 Loaded station config from: {config_path}")
            return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Station config file not found: {config_path}")
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON in station config file: {e}")

def load_and_create_station(
    campaign_id: int,
    config_path: str = "stations/station.json",
    token: str = None,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Load station configuration from JSON and create the station.
    
    Args:
        campaign_id: ID of the parent campaign
        config_path: Path to the station configuration JSON file
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing the created station data with ID
    """
    # Load configuration
    station_config = load_station_config(config_path)
    
    # Validate required fields
    required_fields = ["name", "projectid", "description", "contact_name", "contact_email", "active", "start_date"]
    for field in required_fields:
        if field not in station_config:
            raise ValueError(f"Missing required field '{field}' in station config")
    
    # Display configuration summary
    print(f"📋 Station Configuration Summary:")
    print(f"  Name: {station_config['name']}")
    print(f"  Project ID: {station_config['projectid']}")
    print(f"  Description: {station_config['description'][:100]}...")
    print(f"  Contact: {station_config['contact_name']}")
    print(f"  Active: {station_config['active']}")
    print(f"  Start Date: {station_config['start_date']}")
    
    # Create the station
    station = create_station(
        station_data=station_config,
        campaign_id=campaign_id,
        token=token,
        base_url=base_url
    )
    
    return station

def load_and_create_multiple_stations(
    campaign_id: int,
    config_path: str = "stations/stations.json",
    token: str = None,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> List[Dict[str, Any]]:
    """
    Load multiple station configurations from JSON and create all stations.
    
    Args:
        campaign_id: ID of the parent campaign
        config_path: Path to the stations configuration JSON file
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        List of dictionaries containing the created station data
    """
    # Load configuration
    with open(config_path, 'r', encoding='utf-8') as file:
        stations_config = json.load(file)
    
    created_stations = []
    
    # Handle both single station and multiple stations format
    if "stations" in stations_config:
        station_list = stations_config["stations"]
    else:
        station_list = [stations_config]  # Single station format
    
    print(f"📋 Creating {len(station_list)} station(s)...")
    
    for i, station_config in enumerate(station_list, 1):
        print(f"\n--- Creating Station {i}/{len(station_list)} ---")
        
        try:
            station = create_station(
                station_data=station_config,
                campaign_id=campaign_id,
                token=token,
                base_url=base_url
            )
            created_stations.append(station)
            
        except Exception as e:
            print(f"❌ Failed to create station '{station_config.get('name', 'Unknown')}': {e}")
            continue
    
    return created_stations

In [21]:
print("=== Creating Single Station from Configuration ===")
try:
    station = load_and_create_station(
        campaign_id=campaign_id,
        config_path="stations/station.json",
        token=token
    )
    
    station_id = station['id']
    print(f"\n🎉 Station setup complete!")
    print(f"Station ID: {station_id}")
    
except FileNotFoundError as e:
    print(f"❌ Configuration file error: {e}")
    print("💡 Please create a stations/station.json file with your station details")
except ValueError as e:
    print(f"❌ Configuration error: {e}")
except Exception as e:
    print(f"❌ Station creation failed: {e}")

print("\n" + "="*50)
print("=== Creating Multiple Stations from Configuration ===")
try:
    stations = load_and_create_multiple_stations(
        campaign_id=campaign_id,
        config_path="stations/stations.json",
        token=token
    )
    
    print(f"\n🎉 Created {len(stations)} station(s) successfully!")
    for station in stations:
        print(f"  • {station.get('name', 'Unknown')} (ID: {station['id']})")
    
except FileNotFoundError as e:
    print(f"❌ Configuration file error: {e}")
    print("💡 Please create a stations/stations.json file with your station details")
except Exception as e:
    print(f"❌ Multiple stations creation failed: {e}")

=== Creating Single Station from Configuration ===
📄 Loaded station config from: stations/station.json
📋 Station Configuration Summary:
  Name: Cow Bayou near Mauriceville
  Project ID: SETx-UIFL Beaumont
  Description: Beaumont Run stream gauge at Cow Bayou...
  Contact: Nick Brake
  Active: True
  Start Date: 2025-06-02T14:42:00+0000
✅ Station created successfully!
Station ID: 14
Station Name: Cow Bayou near Mauriceville
Project ID: SETx-UIFL Beaumont
Contact: Nick Brake

🎉 Station setup complete!
Station ID: 14

=== Creating Multiple Stations from Configuration ===
❌ Configuration file error: [Errno 2] No such file or directory: 'stations/stations.json'
💡 Please create a stations/stations.json file with your station details


## CSV Data Upload Function Documentation

### Overview

The `upload_csv_data` function provides a streamlined way to upload sensor and measurement data to the Upstream platform via CSV files. This function handles file validation, authentication, and provides detailed feedback on the upload process.

### Function Signature

```python
def upload_csv_data(
    campaign_id: int,
    station_id: int,
    sensors_file_path: str,
    measurements_file_path: str,
    token: str,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
```

### Parameters

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `campaign_id` | `int` | ✅ | Unique identifier for the target campaign |
| `station_id` | `int` | ✅ | Unique identifier for the target station within the campaign |
| `sensors_file_path` | `str` | ✅ | Local file path to the sensors CSV file |
| `measurements_file_path` | `str` | ✅ | Local file path to the measurements CSV file |
| `token` | `str` | ✅ | Authentication token for API access |
| `base_url` | `str` | ❌ | Base URL for the Upstream API (defaults to dev environment) |

### Return Value

Returns a `Dict[str, Any]` containing the upload response data with statistics including:
- Total sensors processed
- Total measurements added to database
- Data processing time

### Features

#### 🔍 **File Validation**
- Automatically checks if both CSV files exist before attempting upload
- Raises `FileNotFoundError` with descriptive messages for missing files

#### 📊 **Progress Tracking**
- Displays upload parameters for verification
- Shows real-time upload status with emoji indicators
- Provides detailed statistics upon completion

#### 🔐 **Secure Upload**
- Uses authenticated requests via the `make_authenticated_request` helper
- Properly formats files for multipart form data upload

#### 🎯 **Error Handling**
- Pre-upload file existence validation
- Clear error messages for troubleshooting

### Usage Example

```python
try:
    result = upload_csv_data(
        campaign_id=123,
        station_id=456,
        sensors_file_path="./data/sensors.csv",
        measurements_file_path="./data/measurements.csv",
        token="your_auth_token_here"
    )
    
    print(f"Upload successful! {result['Total measurements added to database']} measurements added.")
    
except FileNotFoundError as e:
    print(f"File error: {e}")
except Exception as e:
    print(f"Upload failed: {e}")
```

### API Endpoint

The function uploads to the following endpoint:
```
POST {base_url}/api/v1/uploadfile_csv/campaign/{campaign_id}/station/{station_id}/sensor
```

### File Format Requirements

#### Sensors CSV
- Must contain sensor definition data
- Uploaded as `upload_file_sensors` form field

#### Measurements CSV  
- Must contain measurement data corresponding to the sensors
- Uploaded as `upload_file_measurements` form field

### Console Output Example

```
=== Uploading CSV Data ===
Campaign ID: 123
Station ID: 456
Sensors file: ./data/sensors.csv
Measurements file: ./data/measurements.csv
📤 Uploading files...
✅ Upload completed successfully!
📊 Upload Statistics:
  • Sensors processed: 15
  • Measurements added: 1,250
  • Processing time: 2.3s
```

### Dependencies

- `os` - For file existence checking
- `make_authenticated_request` - Custom function for authenticated API calls
- `Dict`, `Any` from `typing` - For type hints

### Error Scenarios

| Error Type | Cause | Solution |
|------------|-------|----------|
| `FileNotFoundError` | CSV file doesn't exist at specified path | Verify file paths are correct |
| Authentication errors | Invalid or expired token | Refresh authentication token |
| API errors | Server issues or invalid parameters | Check campaign/station IDs and API status |

### Best Practices

1. **Validate Data First**: Ensure your CSV files are properly formatted before upload
2. **Check Permissions**: Verify you have write access to the specified campaign/station
3. **Monitor Output**: Pay attention to the upload statistics to confirm expected data volumes
4. **Handle Errors**: Always wrap calls in try-catch blocks for production use
5. **Use Absolute Paths**: Prefer absolute file paths to avoid path resolution issues


In [ ]:

def upload_csv_data(
    campaign_id: int,
    station_id: int,
    sensors_file_path: str,
    measurements_file_path: str,
    token: str,
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Upload sensor and measurement CSV files to Upstream.
    
    Args:
        campaign_id: ID of the target campaign
        station_id: ID of the target station
        sensors_file_path: Path to sensors CSV file
        measurements_file_path: Path to measurements CSV file
        token: Access token
        base_url: Base URL for the API
        
    Returns:
        Upload response data
    """
    upload_url = f"{base_url}/api/v1/uploadfile_csv/campaign/{campaign_id}/station/{station_id}/sensor"
    
    print(f"=== Uploading CSV Data ===")
    print(f"Campaign ID: {campaign_id}")
    print(f"Station ID: {station_id}")
    print(f"Sensors file: {sensors_file_path}")
    print(f"Measurements file: {measurements_file_path}")
    
    # Verify files exist
    if not os.path.exists(sensors_file_path):
        raise FileNotFoundError(f"Sensors file not found: {sensors_file_path}")
    if not os.path.exists(measurements_file_path):
        raise FileNotFoundError(f"Measurements file not found: {measurements_file_path}")
    
    # Prepare files for upload
    with open(sensors_file_path, 'rb') as sensors_file, \
         open(measurements_file_path, 'rb') as measurements_file:
        
        files = {
            'upload_file_sensors': ('sensors.csv', sensors_file, 'text/csv'),
            'upload_file_measurements': ('measurements.csv', measurements_file, 'text/csv')
        }
        
        print("📤 Uploading files...")
        response = make_authenticated_request(
            method="POST",
            url=upload_url,
            token=token,
            files=files
        )
        
        result = response.json()
        print("✅ Upload completed successfully!")
        
        # Display upload statistics
        print(f"📊 Upload Statistics:")
        print(f"  • Sensors processed: {result.get('Total sensors processed', 'N/A')}")
        print(f"  • Measurements added: {result.get('Total measurements added to database', 'N/A')}")
        print(f"  • Processing time: {result.get('Data Processing time', 'N/A')}")
        
        return result


created successfully!


### CSV File Format Examples

#### Sensors CSV Format

Your `sensors.csv` file defines the sensor metadata and should follow this structure:

```csv
alias,variablename,units,postprocess,postprocessscript
temp_sensor_01,Air Temperature,°C,,
humidity_01,Relative Humidity,%,,
pressure_01,Atmospheric Pressure,hPa,,
wind_speed_01,Wind Speed,m/s,true,wind_correction_script
```

**Column Descriptions:**
- `alias`: Unique identifier for the sensor (used as column header in measurements)
- `variablename`: Human-readable description of what the sensor measures
- `units`: Measurement units (e.g., °C, %, hPa, m/s)
- `postprocess`: Boolean flag indicating if post-processing is required
- `postprocessscript`: Name of the post-processing script (if applicable)

#### Measurements CSV Format

Your `measurements.csv` file contains the actual sensor data and should follow this structure:

```csv
collectiontime,Lat_deg,Lon_deg,temp_sensor_01,humidity_01,pressure_01,wind_speed_01
2024-01-15T10:30:00,30.2672,-97.7431,23.5,65.2,1013.25,2.3
2024-01-15T10:31:00,30.2673,-97.7432,23.7,64.8,1013.20,2.1
2024-01-15T10:32:00,30.2674,-97.7433,23.9,64.5,1013.15,1.8
2024-01-15T10:33:00,30.2675,-97.7434,,64.2,1013.10,1.9
```

**Required Columns:**
- `collectiontime`: Timestamp in ISO 8601 format (YYYY-MM-DDTHH:MM:SS)
- `Lat_deg`: Latitude in decimal degrees
- `Lon_deg`: Longitude in decimal degrees

**Sensor Data Columns:**
- Each sensor `alias` from sensors.csv becomes a column header
- Column names must exactly match the sensor aliases
- Empty values are automatically handled (see row 4 in example)

#### Important File Format Notes

⚠️ **Critical Requirements:**
- Each sensor `alias` from sensors.csv becomes a column in measurements.csv
- `collectiontime`, `Lat_deg`, and `Lon_deg` are required columns in measurements.csv
- Empty values are handled automatically by the system
- Maximum file size is **500 MB per file**
- Use UTF-8 encoding for both files
- Timestamps should be in UTC or include timezone information

📝 **Best Practices:**
- Keep sensor aliases short but descriptive
- Use consistent naming conventions (e.g., `sensor_type_number`)
- Ensure measurement values match the units specified in sensors.csv
- Include all sensors in measurements.csv even if some readings are missing

### Additional Helper Functions

#### Get Available Campaigns

```python
def get_campaigns(
    token: str, 
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Get list of available campaigns.
    
    Args:
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing campaign data
    """
    url = f"{base_url}/api/v1/campaigns"
    response = make_authenticated_request("GET", url, token)
    return response.json()
```

#### Get Campaign Stations

```python
def get_stations(
    campaign_id: int, 
    token: str, 
    base_url: str = "https://upstream-dso.tacc.utexas.edu/dev"
) -> Dict[str, Any]:
    """
    Get list of stations for a specific campaign.
    
    Args:
        campaign_id: ID of the target campaign
        token: Authentication token
        base_url: Base URL for the API
        
    Returns:
        Dictionary containing station data for the campaign
    """
    url = f"{base_url}/api/v1/campaigns/{campaign_id}/stations"
    response = make_authenticated_request("GET", url, token)
    return response.json()
```

#### Usage Example: Exploring Available Resources

```python
# Example: List available campaigns and stations
print("=== Available Campaigns ===")
campaigns = get_campaigns(token)
print(json.dumps(campaigns, indent=2))

print("\n=== Available Stations ===")
stations = get_stations(CAMPAIGN_ID, token)
print(json.dumps(stations, indent=2))

# Find the right campaign and station IDs
for campaign in campaigns.get('data', []):
    print(f"Campaign: {campaign['name']} (ID: {campaign['id']})")
    
    # Get stations for this campaign
    campaign_stations = get_stations(campaign['id'], token)
    for station in campaign_stations.get('data', []):
        print(f"  └── Station: {station['name']} (ID: {station['id']})")
```

#### Helper Function Features

🔍 **Campaign Discovery:**
- List all campaigns you have access to
- View campaign metadata and descriptions
- Identify the correct campaign ID for your data

🏗️ **Station Management:**
- List all stations within a campaign
- View station details and locations
- Find the appropriate station ID for your sensors

💡 **Integration Tip:**
Use these helper functions before uploading data to ensure you're targeting the correct campaign and station IDs.


In [ ]:
def get_campaigns(token: str, base_url: str = "https://upstream-dso.tacc.utexas.edu/dev") -> Dict[str, Any]:
    """Get list of available campaigns."""
    url = f"{base_url}/api/v1/campaigns"
    response = make_authenticated_request("GET", url, token)
    return response.json()

def get_stations(campaign_id: int, token: str, base_url: str = "https://upstream-dso.tacc.utexas.edu/dev") -> Dict[str, Any]:
    """Get list of stations for a campaign."""
    url = f"{base_url}/api/v1/campaigns/{campaign_id}/stations"
    response = make_authenticated_request("GET", url, token)
    return response.json()

# Example: List available campaigns and stations
"""
print("=== Available Campaigns ===")
campaigns = get_campaigns(token)
print(json.dumps(campaigns, indent=2))

print("=== Available Stations ===")
stations = get_stations(CAMPAIGN_ID, token)
print(json.dumps(stations, indent=2))
"""

## 7. Best Practices

1. **File Preparation:**
   - Validate your CSV files before upload
   - Ensure sensor aliases match between files
   - Use consistent timestamp formats

2. **Error Handling:**
   - Always wrap API calls in try-catch blocks
   - Check file existence before upload
   - Validate response status codes

3. **Security:**
   - Never hardcode credentials in notebooks
   - Store tokens securely
   - Use environment variables for sensitive data

4. **Performance:**
   - Keep files under 500 MB for optimal performance
   - Use batch uploads for large datasets
   - Monitor upload progress and statistics